# 🎓 Supervised Fine-Tuning (SFT) - Complete Beginner's Tutorial

## Welcome! 👋

This notebook will teach you **everything you need to know** about fine-tuning large language models (LLMs) like Llama 3.

By the end of this tutorial, you will understand:
- ✅ What is a pre-trained language model?
- ✅ What is fine-tuning and why do we need it?
- ✅ What is LoRA and why is it better than full fine-tuning?
- ✅ How tokenization works
- ✅ How training happens step-by-step
- ✅ What all the hyperparameters mean
- ✅ How to evaluate your model
- ✅ How to use your fine-tuned model

**No machine learning experience required!** We'll explain every concept from the ground up.

---

# Part 1: Understanding the Basics

## 🧠 What is a Language Model?

### Simple Analogy:
Imagine you're learning to predict the next word in a sentence by reading millions of books:

```
"The capital of Nigeria is ___"
→ Model predicts: "Abuja" (because it saw this pattern many times)

"How to register a business ___"
→ Model predicts: "in Nigeria" (learned from similar texts)
```

That's what a language model does! It learns patterns from text and predicts what comes next.

### What is Llama 3?
- **Llama 3**: A powerful language model made by Meta
- **8B version**: 8 billion parameters (think: 8 billion "weights" that make decisions)
- **Pre-trained**: Already learned from trillions of words from the internet
- **General knowledge**: Knows about many topics (geography, science, history, etc.)
- **But**: Not specialized in your specific domain (Nigerian government services)

---

## 🎯 What is Fine-Tuning?

### The Problem:
Llama 3 is great at general knowledge, but if you ask it about specific Nigerian government services, it might not be accurate.

### The Solution:
**Fine-tuning** = Teaching the model to become an expert in your specific domain

### How it works:
1. Start with Llama 3 (general knowledge model)
2. Show it examples from your domain (Nigerian government Q&A)
3. Model adjusts its weights to specialize in this domain
4. Result: A model that's great at answering Nigerian government questions

### Analogy:
```
Pre-trained model ≈ A medical student with 4 years of general training
Fine-tuning ≈ 6 months of specialization in cardiology
Fine-tuned model ≈ A cardiologist (general knowledge + specialization)
```

---

## ⚡ Why LoRA? (Low-Rank Adaptation)

### The Problem with Full Fine-Tuning:
- Llama 3 has **8 billion parameters** (adjustable values)
- Training all 8B parameters takes **days** and uses **100+ GB GPU memory**
- Very expensive! ❌

### What is LoRA?
LoRA is a clever trick:
- Instead of modifying all 8B parameters, we add **small adapter layers**
- Only train ~42M parameters (0.5% of total)
- Result: **Same quality but 2x faster and 60% less memory** ✅

### How LoRA Works (Simplified):
```
Traditional: Update big matrix (8000×8000) - SLOW
LoRA: Update small matrices (8000×16) + (16×8000) - FAST

Combined effect is similar to updating the big matrix!
```

### Memory Savings:
```
Full fine-tuning: 40 GB memory
LoRA: 16 GB memory (4x savings!)
LoRA + 4-bit: 8 GB memory (5x savings!)
```

---

## 🔢 What is 4-bit Quantization?

### Problem: Model weights take lots of memory
```
Weight stored as: 32-bit number = 0.123456789 (32 bits = 4 bytes)
Model with 8B parameters = 8B × 4 bytes = 32 GB
```

### Solution: Use smaller numbers (4-bit instead of 32-bit)
```
Weight stored as: 4-bit number ≈ 0.1 (4 bits = 0.5 bytes)
Model with 8B parameters = 8B × 0.5 bytes = 4 GB
Memory savings: 8x! 🎉
```

### Why does this work?
- Most weights don't need high precision
- 4-bit numbers are "close enough" for inference
- Minimal quality loss with slight accuracy trade-off
- Like taking a photo in 4-bit color instead of 32-bit - you lose some gradients but image is still recognizable

---

# Part 2: Understanding Tokenization

## 🔤 What is a Token?

Language models don't understand words—they understand **numbers**.

**Tokenization** = Converting text into numbers

### Example:
```
Text: "Hello Nigeria"
       ↓ (tokenization)
Tokens: [15339, 29878]  (These are just numbers!)
```

### What is a token?
A token is typically:
- A word: "hello" → 15339
- Part of a word: "ing" → 29878 (from "running")
- Punctuation: "." → 29889
- Space: " " → 1 (special token)

### Vocabulary:
Llama 3 has a **vocabulary of 128,000 tokens**
- Each token maps to a unique number (0-127,999)
- The tokenizer has a lookup table for mapping

### Tokenization Example:
```python
Text: "How to register a business?"
Tokens: [29882, 304, 8369, 263, 12881, 29973]
         ↑     ↑   ↑     ↑   ↑      ↑
         How   to  reg   a   bus    ?
```

### Why tokenization matters for fine-tuning:
1. Models can only process tokens, not text
2. Longer text = more tokens = more memory
3. This is why `MAX_SEQ_LENGTH=1024` is important (max 1024 tokens per example)
4. Language varies: English text might tokenize differently than Yoruba

### Tokens vs Words:
```
English sentence: "I'm going to Nigeria"
Words: 5 words
Tokens: ~8 tokens (contractions and punctuation create extra tokens)

Rule of thumb: 1 token ≈ 0.75 words
1000 tokens ≈ 750 words
```

---

# Part 3: Understanding Training

## 📚 How Training Works

### The Training Loop (Simplified):
```
1. Start with pre-trained Llama 3 model
2. Take one batch of training examples (e.g., 8 Q&A pairs)
3. Model makes predictions on these examples
4. Calculate LOSS = how wrong the predictions are
5. Update model weights to reduce loss
6. Repeat 60 times (or more for larger training)
7. Model gets better at answering questions!
```

### What is Loss?
**Loss** measures how wrong your model's predictions are:
```
Example:
Question: "What is the capital of Nigeria?"
Reference answer: "Abuja"

Model's first attempt: "Lagos"
Loss = VERY HIGH (completely wrong) ❌

After 10 training steps:
Model predicts: "Abuja or Lagos"
Loss = MEDIUM (partially correct)

After 60 training steps:
Model predicts: "Abuja"
Loss = LOW (correct!) ✅
```

### Loss Formula (Simplified):
```
Loss = Average difference between model predictions and correct answers

Lower loss = better model
Higher loss = worse model
```

---

## 🔄 Batch Size and Gradient Accumulation

### What is a Batch?
Instead of training on one example at a time, we train on multiple examples together:
```
Example 1: Q: "How to register a business?" A: "Complete the FIRS form..."
Example 2: Q: "What is CAC?" A: "Corporate Affairs Commission..."
Example 3: Q: "Cost of CAC registration?" A: "₦50,000..."
           ↓
        (Batch of 3)
```

### Why batch training?
1. **More stable learning**: Average loss across multiple examples
2. **Faster**: GPU can process multiple examples at once
3. **Better generalization**: Model learns from diverse examples

### Batch Size Configuration:
```python
PER_DEVICE_BATCH_SIZE = 2        # Process 2 examples at a time
GRADIENT_ACCUMULATION_STEPS = 4  # Accumulate for 4 batches

Effective batch size = 2 × 4 = 8 examples before updating weights
```

### Why Gradient Accumulation?
Your GPU might not have enough memory for batch size 8, so we:
1. Process batch of 2 examples
2. Calculate gradients (not weights yet)
3. Store gradients in memory
4. Repeat 4 times
5. Add up all gradients
6. Update weights once

**Result**: Same effect as batch size 8, but uses 4x less memory!

### Memory vs Speed Trade-off:
```
Large batch size (32):   Fast ⚡ but needs more memory 💾
Small batch size (2):    Slow 🐢 but uses less memory 💾
With accumulation (2×4): Medium speed ⚡ and memory 💾
```

---

## 🎛️ Understanding Learning Rate

### What is Learning Rate?
**Learning rate** = How big a step to take when updating weights

### Analogy: Finding the bottom of a valley
```
Too high learning rate (LR = 0.1):     Too low learning rate (LR = 0.00001):
      ^                                  ^    
      |  ↗️  ↙️  ↗️                     |       ... ... ...
      |  (jumps too far)                 |   (takes forever)
      |                                  |
    Result: Diverges, never finds bottom   Result: Finds bottom but takes ages

Just right learning rate (LR = 0.0002):  
      ^    
      |  \→ → → ✓
      |  (smooth descent to bottom)
      |
    Result: Converges smoothly to good solution
```

### Learning Rate Schedule:
```python
LEARNING_RATE = 2e-4  # Start at 0.0002
WARMUP_STEPS = 5      # Gradually increase for first 5 steps
```

Why warmup?
- Model is randomly initialized and unstable
- Start with small LR, gradually increase to 2e-4
- Helps training stabilize faster

### Common Learning Rates:
```
Fine-tuning: 2e-4 to 5e-5 (small, because model already trained)
Pre-training: 1e-3 to 3e-4 (larger, training from scratch)
Classification: 1e-3 to 1e-4 (varies by dataset size)
```

---

# Part 4: Understanding Hyperparameters

## 📊 Complete Hyperparameter Breakdown

### Model Architecture Parameters:

```python
MODEL_NAME = "unsloth/llama-3.1-8b-Instruct-bnb-4bit"
# ├─ "unsloth": Optimized version for faster training
# ├─ "llama-3.1": Model type
# ├─ "8b": 8 billion parameters
# ├─ "Instruct": Fine-tuned for following instructions
# └─ "bnb-4bit": Uses 4-bit quantization

MAX_SEQ_LENGTH = 1024
# Maximum tokens in one training example
# If example is longer, it gets truncated
# If shorter, it gets padded (filled with special tokens)
# ~750 words ≈ 1000 tokens (rule: 1 token ≈ 0.75 words)
# Larger = better but uses more memory

LOAD_IN_4BIT = True
# Load model in 4-bit precision
# 8x memory savings with minimal quality loss
```

### LoRA Parameters:

```python
LORA_R = 16
# Rank of LoRA adapters
# Higher R = more parameters to train, better quality but slower
# Common values: 8, 16, 32, 64
# For small datasets: 8-16
# For large datasets: 32-64

LORA_ALPHA = 16
# Scaling factor for LoRA updates
# Usually set equal to LORA_R
# Affects how much LoRA contributes to final model
# Effective scaling = LORA_ALPHA / LORA_R = 1.0 (neutral)

LORA_DROPOUT = 0
# Regularization: randomly drops out connections during training
# 0 = no dropout
# 0.05 = drop 5% of connections (prevents overfitting)
# For small datasets, use 0.05 to prevent overfitting
# For large datasets, can use 0 to train faster
```

### Training Schedule Parameters:

```python
MAX_STEPS = 60
# Maximum number of training steps
# 1 step = 1 weight update
# For testing: 60-200 steps
# For real training: 500-5000 steps (depends on dataset size)
# Formula: steps = (num_examples × epochs) / batch_size

PER_DEVICE_BATCH_SIZE = 2
# Examples processed per GPU before weight update
# Larger = faster but needs more memory
# Smaller = slower but uses less memory
# Common: 2-8 for fine-tuning

GRADIENT_ACCUMULATION_STEPS = 4
# Accumulate gradients for N batches before updating weights
# Effective batch = 2 × 4 = 8
# Trade: GPU memory for computational efficiency
# Higher = more memory efficient but slower convergence
```

### Optimization Parameters:

```python
LEARNING_RATE = 2e-4  (0.0002)
# Step size for weight updates
# Too high: Training diverges (loss increases)
# Too low: Training is very slow
# For fine-tuning: 1e-4 to 5e-4 is typical

WARMUP_STEPS = 5
# Gradually increase LR for first N steps
# Helps stabilize training
# Typical: 5-10% of total steps
# For 60 steps: 5 warmup steps is good

WEIGHT_DECAY = 0.01
# L2 regularization to prevent overfitting
# Adds penalty for large weights
# Typical: 0 (none) to 0.1
# Higher = more regularization = underfitting risk

LR_SCHEDULER_TYPE = "linear"
# How to adjust learning rate during training
# Options: "linear", "cosine", "constant"
# Linear: Decrease LR linearly from peak to near 0
# Cosine: Smooth decrease following cosine curve
# Constant: Keep LR fixed throughout

OPTIM = "adamw_8bit"
# Optimization algorithm (AdamW)
# "8bit": Uses 8-bit precision for optimizer states
# Saves memory without hurting convergence
```

### Precision Parameters:

```python
FP16 = True   # 16-bit floating point (older GPUs like T4)
BF16 = False  # Brain Float 16 (newer GPUs like A100)
# Only one should be True!
# FP16: Good precision but can be unstable
# BF16: More stable, better for training
# 2x faster than FP32, half the memory
```

---

## ⚙️ How to Choose Hyperparameters

### For Your First Fine-Tuning:

| Parameter | Value | Why? |
|-----------|-------|------|
| LORA_R | 8-16 | Smaller = faster for experimentation |
| MAX_STEPS | 60-200 | Quick test to verify training works |
| PER_DEVICE_BATCH_SIZE | 2-4 | Smaller = less memory |
| LEARNING_RATE | 2e-4 | Safe default for fine-tuning |
| WARMUP_STEPS | 5-10 | ~10% of total steps |

### For Production:

| Parameter | Value | Why? |
|-----------|-------|------|
| LORA_R | 32-64 | Larger = better quality |
| MAX_STEPS | 1000-5000 | More training = better results |
| PER_DEVICE_BATCH_SIZE | 4-8 | Larger = faster convergence |
| LEARNING_RATE | 1e-4 to 5e-4 | Experiment and pick best |
| WARMUP_STEPS | 100-500 | ~10% of total steps |

---

# Part 5: Data Preparation

## 📚 Training Data Format

### Required Format (JSON):
```json
[
  {
    "question": "How do I register my business in Nigeria?",
    "answer": "To register your business, follow these steps: 1. Go to CAC website, 2. Fill the form, 3. Pay fees, 4. Get certificate",
    "agency": "CAC"
  },
  {
    "question": "What is the cost of business registration?",
    "answer": "The cost is ₦50,000 for business registration with CAC",
    "agency": "CAC"
  }
]
```

### Why this format?
- **question**: What the user asks
- **answer**: The correct response (model learns to generate this)
- **agency**: For tracking/filtering (optional)

---

## 🔀 Train/Validation/Test Split

### Why split data?
```
Training set (80%):    Model learns from these examples
                       Loss decreases as model trains
                       ↓
Validation set (10%):  Check if model generalizes
                       If val_loss >> train_loss → OVERFITTING
                       ↓
Test set (10%):        Final evaluation (never seen during training)
                       True measure of model quality
```

### What is Overfitting?
```
Good model:                    Overfitted model:
Training loss: 0.5             Training loss: 0.1 (very low)
Validation loss: 0.52          Validation loss: 2.5 (very high)
Test loss: 0.51                Test loss: 2.4
→ Generalizes well             → Memorized training data
```

### How much data do you need?
```
100-500 examples:    Quick experiment/proof of concept
500-2000 examples:   Small fine-tune, acceptable quality
2000-10000:          Good fine-tune with solid results
10000+:              Excellent fine-tune, strong specialization
```

### Data Quality Matters More Than Quantity:
```
500 high-quality Q&A pairs > 5000 low-quality pairs

High-quality means:
- Accurate answers
- Clear questions
- Diverse examples
- Correct grammar
```

---

# Part 6: Evaluation Metrics

## 📊 Understanding Loss

### Training Loss vs Validation Loss:
```
Step 1:   Training loss: 2.5, Validation loss: 2.6
Step 30:  Training loss: 0.8, Validation loss: 0.85
Step 60:  Training loss: 0.3, Validation loss: 0.35

Good: Both decrease together
```

### Red Flags:
```
Training loss: 0.1, Validation loss: 5.0 → OVERFITTING
Training loss: 0.5, Validation loss: 0.5, but not decreasing → NOT TRAINING
Training loss: increases → LEARNING RATE TOO HIGH
```

---

## 🔴 ROUGE Scores (Advanced Evaluation)

### What is ROUGE?
**ROUGE** = Recall-Oriented Understudy for Gisting Evaluation

Measures overlap between model response and reference answer:

```
Reference: "The capital of Nigeria is Abuja located in central Nigeria"
Response:  "Nigeria's capital is Abuja in central Nigeria"

Matching words: capital, of, is, Abuja, located, in, Nigeria
ROUGE Score: measures how many matching words
```

### ROUGE Types:

```python
ROUGE-1: Unigram (single word) overlap
# Does response contain same words as reference?
# Reference: "Abuja is capital"
# Response:  "capital is Abuja"  
# ROUGE-1: 3/3 = 1.0 (all words match)

ROUGE-2: Bigram (two-word pairs) overlap
# Does response contain same phrases?
# Reference: "Abuja is capital"
# Response:  "capital is Abuja"
# ROUGE-2: 0/2 = 0.0 (no matching phrases)
# (Different word order = different bigrams)

ROUGE-L: Longest common subsequence
# What's the longest matching sequence?
# Reference: "Abuja is capital"
# Response:  "Abuja is capital"
# ROUGE-L: 1.0 (perfect match)
```

### Interpreting ROUGE Scores (0-1 scale):

```
ROUGE-1 > 0.4:  Good word overlap ✅
ROUGE-1 0.2-0.4: Fair overlap ⚠️
ROUGE-1 < 0.2:  Poor overlap ❌

ROUGE-L > 0.3:  Good sentence similarity ✅
ROUGE-L 0.1-0.3: Fair similarity ⚠️
ROUGE-L < 0.1:  Poor similarity ❌
```

### Important Limitation:
ROUGE only measures **surface-level overlap**, not semantic meaning:

```
Reference: "Nigeria's capital is Abuja"
Response 1: "The capital of Nigeria is Abuja"     → ROUGE-1: 0.80 (high)
Response 2: "Lagos is the largest city in Nigeria" → ROUGE-1: 0.33 (low)

Both are reasonable answers but ROUGE scores differ significantly!
```

---

## 💡 Better Evaluation Methods

### Manual Evaluation:
Read model responses and rate quality 1-5:
```
1 = Completely wrong
2 = Mostly wrong
3 = Partially correct
4 = Mostly correct
5 = Perfect answer

Average rating shows true quality better than ROUGE
```

### Task-Specific Metrics:
```
For Q&A: Did model answer the question?
For Code: Does generated code run without errors?
For Classification: Accuracy on held-out test set
```

---

# Part 7: The Complete Training Workflow

## 🔄 Step-by-Step Process

```
1. SETUP
   └─ Install libraries (Unsloth, Transformers, TRL, etc.)
   └─ Check GPU availability
   └─ Load configuration
   ↓
2. LOAD MODEL
   └─ Download Llama 3 (8B model)
   └─ Load in 4-bit quantization
   └─ Load tokenizer
   ↓
3. ADD LORA
   └─ Add LoRA adapter layers
   └─ Freeze base model weights
   └─ Only ~42M parameters trainable (0.5% of 8B)
   ↓
4. PREPARE DATA
   └─ Load JSON training data
   └─ Convert to chat format
   └─ Split: 80% train, 10% val, 10% test
   ↓
5. TOKENIZE
   └─ Apply chat template
   └─ Convert text to tokens
   └─ Pad/truncate to MAX_SEQ_LENGTH
   ↓
6. CONFIGURE TRAINER
   └─ Set batch size, learning rate, steps
   └─ Choose optimizer and scheduler
   └─ Set up logging
   ↓
7. TRAIN
   └─ Feed batches to model
   └─ Compute loss
   └─ Update LoRA weights
   └─ Repeat for MAX_STEPS times
   ↓
8. EVALUATE
   └─ Run model on validation set
   └─ Calculate ROUGE scores on test set
   └─ Check for overfitting
   ↓
9. SAVE
   └─ Save LoRA adapters (~50MB)
   └─ Save merged model (~16GB)
   └─ Save tokenizer
   ↓
10. DEPLOY
    └─ Use for inference
    └─ Answer questions
    └─ Or upload to Hugging Face
```

---

# Part 8: Common Issues and Debugging

## ⚠️ Problem: Out of Memory (OOM)

### Symptoms:
```
RuntimeError: CUDA out of memory. Tried to allocate X.XX GiB
```

### Solutions (ranked by effectiveness):

```python
1. Reduce PER_DEVICE_BATCH_SIZE:
   2 → 1
   (Uses 50% less memory)

2. Increase GRADIENT_ACCUMULATION_STEPS:
   4 → 8
   (Accumulate for 8 batches instead of 4)

3. Reduce MAX_SEQ_LENGTH:
   1024 → 512
   (Shorter sequences = less memory)

4. Reduce LORA_R:
   16 → 8
   (Smaller adapters = less memory)

5. Use packing=True:
   (Combine short examples to be more memory efficient)

6. Use gradient checkpointing:
   (Already enabled in Unsloth by default)
```

---

## ⚠️ Problem: Training Loss Not Decreasing

### Symptoms:
```
Step 1:  Loss = 2.5
Step 10: Loss = 2.5 (no change!)
Step 30: Loss = 2.5 (still no change!)
```

### Causes and Solutions:

```python
1. Learning rate too low:
   LR = 1e-6 (way too small)
   Solution: Increase to 2e-4 or 5e-4

2. Model not training (weights frozen?):
   Check: Are requires_grad=True for LoRA params?
   Solution: Verify LoRA was added correctly

3. Data format wrong:
   If all examples are identical
   Solution: Check JSON format and data quality

4. Not enough training steps:
   MAX_STEPS = 5 (too few)
   Solution: Increase to at least 60-100
```

---

## ⚠️ Problem: Loss Increasing (Diverging)

### Symptoms:
```
Step 1:  Loss = 2.5
Step 10: Loss = 5.0 (increasing!)
Step 30: Loss = 10.0 (getting worse!)
```

### Causes and Solutions:

```python
1. Learning rate too high:
   LR = 0.1 (way too high)
   Solution: Decrease to 2e-4 or 5e-5

2. Data quality issues:
   Corrupted or invalid examples
   Solution: Check training data format and content

3. Floating point precision issues:
   Solution: Use BF16 instead of FP16 (if GPU supports)
```

---

## ⚠️ Problem: Model Overfitting

### Symptoms:
```
Training loss:   0.1 (very low)
Validation loss: 2.5 (very high)
Test ROUGE: 0.15 (poor)
```

### Solutions:

```python
1. Add dropout:
   LORA_DROPOUT = 0 → 0.05
   (Prevents overfitting)

2. Increase training data:
   More examples = model learns patterns not memorization

3. Add weight decay:
   weight_decay = 0 → 0.01
   (Penalizes large weights)

4. Reduce LORA_R:
   16 → 8
   (Fewer parameters = harder to memorize)

5. Use early stopping:
   Stop training when val_loss stops improving
   (Don't train for MAX_STEPS if not improving)
```

---

# Part 9: Advanced Concepts

## 🔬 What Happens During Training (Deep Dive)

### Inside One Training Step:

```
Input: Batch of 2 examples (effective batch = 8 with accumulation)

Example 1:
  Q: "How to register business?"
  A: "Complete FIRS form at CAC office..."
  
Example 2:
  Q: "What is CAC?"
  A: "Corporate Affairs Commission oversees registration..."

STEP 1: Convert to tokens
  ["How", "to", "register", ...] → [1294, 304, 8369, ...]
  
STEP 2: Forward pass through model
  Tokens go through transformer layers
  Each layer processes the sequence
  Attention mechanisms learn relationships between words
  Output: Predicted tokens
  
STEP 3: Calculate loss
  Compare predicted tokens with ground truth
  If correct: loss = low
  If wrong: loss = high
  Average loss across batch
  
STEP 4: Backward pass
  Calculate how much each weight contributed to the error
  Compute gradients (derivatives) for each parameter
  
STEP 5: Update weights (LoRA only!)
  weight = weight - (learning_rate × gradient)
  Only LoRA weights updated (~42M parameters)
  Base model weights frozen
  
RESULT: Model slightly better at predicting answers
```

---

## 🧬 How LoRA Actually Works

### The Math (Simplified):

```python
# Without LoRA:
output = W × input  # W is 8000×8000 matrix (64M parameters)

# With LoRA:
# Instead of updating W directly, we add a small update:
W_new = W + ΔW

# ΔW is approximated as a product of two smaller matrices:
ΔW ≈ (U × V^T) × scale
#      ^   ^
#      |   └─ 16×8000 = 128K params
#      └───── 8000×16 = 128K params
#        Total: 256K params (0.4% of 64M!)

# Why does this work?
# The update needed during fine-tuning has LOW RANK
# You can represent it with small matrices!
# Similar to PCA - most variance in few dimensions
```

### Visual:
```
Original weight matrix (8000×8000):
┌─────────────────────────┐
│                         │
│   64M parameters        │
│   (Don't update)        │
│                         │
└─────────────────────────┘

LoRA adapter (8000×16 + 16×8000):
┌──────────┐     ┌──────────┐
│ 128K     │  ×  │ 128K     │  = 256K params to train
│ params   │     │ params   │    (Update these!)
└──────────┘     └──────────┘

Result:
output = W×input + (U × V^T × scale × input)
```

---

## 🎯 Different Types of Fine-Tuning

### Full Fine-Tuning:
```
Train all 8B parameters
Pros: Best quality, most flexible
Cons: Slow (days), expensive (100+ GB RAM), requires powerful GPU
When: Large datasets (100K+), unlimited budget
```

### LoRA Fine-Tuning:
```
Train only ~42M parameters (0.5%)
Pros: 2x faster, 60% less memory, good quality
Cons: Slightly lower quality than full fine-tuning
When: Most practical applications, limited resources
```

### QLoRA Fine-Tuning:
```
LoRA + 4-bit quantization
Pros: 8x less memory, still good quality
Cons: Slower than LoRA alone
When: Very limited memory (< 12 GB GPU)
```

### Prompt Tuning:
```
Learn only the prompt prefix (a few tokens)
Pros: Extremely fast, minimal memory
Cons: Limited expressiveness, lower quality
When: Quick experiments, multi-task learning
```

---

# Part 10: After Training - What's Next?

## 💾 Saving Your Model

### Two Options:

### Option 1: Save LoRA Adapters (Recommended)
```python
model.save_pretrained("./lora_adapters")
# Saves: ~50-100 MB
# What it contains:
# - adapter_config.json (LoRA configuration)
# - adapter_model.bin (LoRA weights)
# - tokenizer.model (vocabulary)

# To use later:
from peft import PeftModel
model = PeftModel.from_pretrained(
    "unsloth/llama-3.1-8b-Instruct-bnb-4bit",
    "./lora_adapters"
)
```

**Pros:**
- Tiny file size (perfect for sharing)
- Easy to switch between different LoRA adapters
- Fast to load

**Cons:**
- Need base model at inference time
- Not standalone

### Option 2: Save Merged Model
```python
model.save_pretrained_merged(
    "./merged_model",
    tokenizer
)
# Saves: ~16 GB (full model size)

# To use later:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("./merged_model")
```

**Pros:**
- Standalone model (no base model needed)
- Ready for deployment
- Can upload to Hugging Face directly

**Cons:**
- Large file size (impractical for many scenarios)
- Takes time to save/load

---

## 🚀 Using Your Fine-Tuned Model

### Inference (Making Predictions):
```python
# Prepare model for inference
from unsloth import FastLanguageModel
FastLanguageModel.for_inference(model)

# Create prompt
prompt = """Question: How to register a business in Nigeria?
Answer: """

# Generate response
inputs = tokenizer(
    prompt,
    return_tensors="pt"
).to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=512,  # Generate up to 512 tokens
    temperature=0.7,      # Randomness (0=deterministic, 1=very random)
    top_p=0.9,           # Nucleus sampling (keep top 90% probability)
    do_sample=True       # Use sampling instead of greedy
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)
```

### Generation Parameters Explained:
```python
max_new_tokens=512
  # Maximum tokens to generate
  # Longer = slower but more complete responses
  # Shorter = faster but truncated

temperature=0.7
  # Controls randomness (0-1)
  # 0.0 = Always pick highest probability (deterministic)
  # 0.7 = Medium randomness (balanced)
  # 1.0+ = Very random (creative but potentially nonsensical)

top_p=0.9
  # Nucleus sampling: only consider top 90% probability tokens
  # Lower = more focused, coherent responses
  # Higher = more diverse, sometimes strange responses

top_k=50
  # Only consider top 50 most likely tokens
  # Prevents very unlikely token selection

do_sample=True
  # Use sampling instead of greedy decoding
  # True = More natural, varied responses
  # False = More deterministic, repetitive
```

---

## 🌐 Deploying to Hugging Face Hub

### Why upload?
```
✅ Share with others
✅ Permanent storage
✅ Easy integration with other projects
✅ Version control for models
```

### Steps:
```python
# 1. Install and login
from huggingface_hub import login
login()  # Paste your HF token

# 2. Upload LoRA adapters
model.push_to_hub(
    "your_username/model_name",
    private=False  # True = only you can access
)
tokenizer.push_to_hub("your_username/model_name")

# 3. Now anyone can load:
from peft import PeftModel
model = PeftModel.from_pretrained(
    "unsloth/llama-3.1-8b-Instruct-bnb-4bit",
    "your_username/model_name"
)
```

---

# Part 11: Summary & Quick Reference

## 📋 Key Concepts at a Glance

| Concept | Simple Explanation | When Important |
|---------|-------------------|----------------|
| **Pre-training** | Model learns from internet data | Foundation for fine-tuning |
| **Fine-tuning** | Specialize model for your domain | Main goal |
| **LoRA** | Train only 0.5% parameters | Saves time and memory |
| **Quantization** | Use smaller numbers (4-bit) | Reduces memory usage |
| **Tokenization** | Convert text to numbers | Model input format |
| **Batch Size** | Examples per update | Memory vs speed trade-off |
| **Learning Rate** | Step size for updates | Controls convergence |
| **Warmup** | Gradually increase LR | Stabilizes training |
| **Overfitting** | Model memorizes training data | Check with validation set |
| **ROUGE** | Measure response quality | Evaluate after training |

---

## 🎯 Best Practices

### Before Training:
- ✅ Prepare 500-5000 high-quality Q&A pairs
- ✅ Verify data format (question-answer JSON)
- ✅ Check GPU memory with `nvidia-smi`
- ✅ Start with smaller batch sizes if unsure

### During Training:
- ✅ Monitor training/validation loss graphs
- ✅ Watch for divergence (loss increasing)
- ✅ Check validation loss to catch overfitting
- ✅ Save checkpoints periodically

### After Training:
- ✅ Evaluate on test set (never seen during training)
- ✅ Manual evaluation of sample responses
- ✅ Calculate ROUGE scores
- ✅ Compare to baseline model
- ✅ Save both LoRA adapters and merged model

---

## 🚨 Common Mistakes to Avoid

```
❌ Using same data for train AND validation
   → You can't detect overfitting!
   ✅ Always keep test data separate

❌ Training for only 10 steps
   → Model hasn't learned much
   ✅ Minimum 60 steps, better with 500+

❌ Setting learning rate to 0.1
   → Training will diverge immediately
   ✅ Use 2e-4 to 5e-4 for fine-tuning

❌ Ignoring out-of-memory errors
   → Training will crash
   ✅ Reduce batch size or max_seq_length

❌ Not saving model checkpoints
   → If training crashes, lose everything
   ✅ Save model every 100 steps

❌ Using low-quality training data
   → Model learns garbage
   ✅ Quality matters more than quantity
```

---

## 📊 Expected Results

### Training Loss Progression (for 60 steps):
```
Step 1:   Loss = 2.5  (random model)
Step 10:  Loss = 1.8  (20% improvement)
Step 30:  Loss = 0.8  (65% improvement)
Step 60:  Loss = 0.4  (84% improvement)

This is normal! Loss improvement is logarithmic.
```

### Expected ROUGE Scores (after 60 steps):
```
Small dataset (100 examples):  ROUGE-1 ≈ 0.25-0.35
Medium dataset (1000 examples): ROUGE-1 ≈ 0.35-0.50
Large dataset (10K examples):   ROUGE-1 ≈ 0.50-0.65

Note: These are just benchmarks, actual results depend on data quality
```

---

## 🔧 Hyperparameter Tuning Guide

### If training is too slow:
```
1. Reduce MAX_SEQ_LENGTH (1024 → 512)
2. Increase PER_DEVICE_BATCH_SIZE (2 → 4)
3. Reduce LORA_R (16 → 8)
4. Increase GRADIENT_ACCUMULATION_STEPS (4 → 1)
```

### If model quality is poor:
```
1. Increase training data (more examples)
2. Increase MAX_STEPS (60 → 500)
3. Increase LORA_R (8 → 32)
4. Reduce LEARNING_RATE (2e-4 → 5e-5)
5. Increase WARMUP_STEPS (5 → 50)
```

### If out of memory:
```
1. Reduce PER_DEVICE_BATCH_SIZE (2 → 1)
2. Increase GRADIENT_ACCUMULATION_STEPS (4 → 8)
3. Reduce MAX_SEQ_LENGTH (1024 → 512)
4. Reduce LORA_R (16 → 8)
```

---

# 🎓 Conclusion

## You now understand:

✅ **What** fine-tuning is and why it's useful

✅ **Why** LoRA makes it practical

✅ **How** tokenization works

✅ **What** each hyperparameter controls

✅ **How** the training process works step-by-step

✅ **Why** we split data into train/validation/test

✅ **How** to evaluate your model

✅ **What** common issues to watch for

---

## Next Steps:

1. **Prepare your data**: Collect 500-5000 high-quality Q&A pairs
2. **Run the original notebook**: Use the accompanying `sft_finetuning.ipynb`
3. **Experiment**: Try different hyperparameters and see the effects
4. **Evaluate**: Test your model on held-out examples
5. **Deploy**: Share your model with others or integrate into applications

---

## Helpful Resources:

- **Hugging Face Documentation**: https://huggingface.co/docs
- **Unsloth GitHub**: https://github.com/unslothai/unsloth
- **PEFT (Parameter-Efficient Fine-Tuning)**: https://github.com/huggingface/peft
- **Llama 3 Model Card**: https://huggingface.co/meta-llama/Llama-3-8b

---

## Good luck with your fine-tuning! 🚀

Remember: The key to good results is:
1. **High-quality data** (more important than fancy hyperparameters)
2. **Proper evaluation** (understand how well your model really performs)
3. **Patience with experimentation** (small changes can have big effects)
4. **Reading error messages** (they usually tell you exactly what's wrong)

Happy fine-tuning! 🎉